In [1]:
from parameter import *
from dspike_formulas import *
import numpy as np
from matplotlib import pyplot as pl
import shelve

In [5]:
path = "/Users/marf 1/Desktop/PhD Temp/Planning Sn 117-122/real data -intern norm/"
df = pd.read_csv(str(path + "2015_03_25_1367_raw_ratio_corr.csv"))
df_new = df.ix[:,"112":"124"]
df_new = df_new.drop(["117_2", "118_2", "119_2", "122_2"], axis=1)

# Isotope used for spike calculation [[denominator],[nominator1, nominator2, nominator3]]
spike112_117_ls = [["116"],["112", "117", "124"]]
spike112_118_ls = [["116"],["112", "118", "124"]]
spike117_124_ls = [["116"],["117", "122", "124"]]
spike117_122_1_ls = [["116"],["117", "122", "124"]]
spike117_122_2_ls = [["117"],["116", "120", "122"]]
spike117_120_ls = [["116"],["117", "120", "124"]]
spike112_118_2_ls = [["116"],["112", "118", "120"]]
spike117_122_3_ls = [["117"],["118", "120", "122"]]
spike120_124_ls = [["117"],["120", "122", "124"]]
spike120_122_ls = [["118"],["120", "122", "124"]]
spike118_122_ls = [["118"],["120", "122", "124"]]
spike116_122_ls = [["118"],["116", "122", "124"]]
# Define Model Parameter#
#*** Natural Fractionation ***#
fnat_sim = -1
#*** Instrumental Fractionation ***#
fins_sim =  2.2
#*** Sample/Spike Ratio ***#
mix = 0.5
mix_2 = [0.02, 0.05, 0.1, 0.2, 0.5, 0.8, 0.9, 0.95, 0.98]
# Dampening of simulated mixtures
damp = 0.9

In [3]:
# p (Sample-Spike Mix) and q (Spike1-Spike2-Mix) range used for calculation
p_rang = np.linspace(0.1, 0.9, 33)
#p_rang1 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
q_rang = np.linspace(0.1, 0.9, 33)
#q_rang1 = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [6]:
# Spike simulation, for each p in Sample-Spike Mix the Errors of the q_range are determined and returned in a dictionary 
z = collections.OrderedDict()
for p in p_rang:
    counter = 0
    sim1_q = spike_sim_q_range(q_rang,spike_116,spike_122,Sn_meas_obj,df_new,Sn_mass_obj,spike116_122_ls,p,fnat_sim,fins_sim,damp,3,6,-0.1,-2,'z')
    q_ls, z_ls = error_vs_q(sim1_q, 'z')
    z.update({p: collections.OrderedDict()})
    for q_value in q_ls:
        z[p].update({q_value : z_ls[counter]})
        counter+=1

In [7]:
# converts dictionary into dataframe with p (row) vs q (column)
z_df_117_122_2 = pd.DataFrame.from_dict(z,"index")
print z_df_117_122_2
print z_df_117_122_2.min() # Minimum value for each q
print z_df_117_122_2.idxmin() # Minimum index(p) for each q
z_matrix_1 = z_df_117_122_2.as_matrix() # Matrix q vs p
z_matrix_1 = z_matrix_1.transpose() # Returns array with axes transposed -> matrix p vs q
mini = z_matrix_1.min() # absolute Minimum of matrix
#db=shelve.open("df_d_120.dbm")
#db["z_df_117_122_2"]=z_df_117_122_2


             0.100        0.125        0.150        0.175        0.200  \
0.100   775.962955   741.383174   714.650924   692.234666   672.449185   
0.125   670.922158   639.377730   615.650510   596.211753   579.381346   
0.150   604.212104   574.020514   551.872810   534.127350   519.053751   
0.175   559.667650   529.793358   508.363635   491.545546   477.520884   
0.200   529.238517   498.964677   477.670123   461.270346   447.830325   
0.225   508.484914   477.266344   455.675312   439.324863   426.138579   
0.250   494.776591   462.165147   439.932601   423.343616   410.158018   
0.275   486.475137   452.077836   428.909370   411.842368   398.451530   
0.300   482.526150   445.979485   421.609498   403.853699   390.080441   
0.325   482.240814   443.193559   417.370924   398.731482   384.415573   
0.350   485.172472   443.273056   415.750883   396.039957   381.029986   
0.375   491.044197   445.930488   416.458139   395.488118   379.635457   
0.400   499.705507   450.995688   419.

In [8]:
# Creates contour lines with spacing of 1-25% of the optimum value
print mini
contour_spacing = []
for i in range(1, 26):
    x = mini + ((i/float(100)) * mini)
    contour_spacing.append(x) 
print contour_spacing

270.242700321
[272.94512732455854, 275.64755432777196, 278.34998133098543, 281.05240833419884, 283.75483533741232, 286.45726234062579, 289.1596893438392, 291.86211634705268, 294.56454335026615, 297.26697035347956, 299.96939735669304, 302.67182435990645, 305.37425136311992, 308.0766783663334, 310.77910536954681, 313.48153237276028, 316.18395937597376, 318.88638637918717, 321.58881338240064, 324.29124038561406, 326.99366738882753, 329.696094392041, 332.39852139525442, 335.10094839846789, 337.80337540168136]


In [10]:
# Creates contour plot
fig = pl.figure()
pl.axes([0.025, 0.025, 0.95, 0.95])
pl.contour(p_rang, q_rang, z_matrix_1, contour_spacing, colors='black', linewidth=.5)
#pl.clabel(C, inline=1, fontsize=10)
pl.xlabel('p - Proportion of Sample Double-Spike-Sample Mix')
pl.ylabel('q - Proportion of Sn116 in Sn116-Sn122')
pl.title('Error plot Sn116-122,124 d=118')
pl.plot(0.300, 0.875, marker='x', color="red" ,ls='')
pl.text(0.300, 0.88, str(np.round(mini,2)))
pl.text(0.45, -0.03, 'Optimum: p = 0.300, q = 0.875, ' + r'$\alpha$' + '= 270.24 ppm')
pl.show(fig)
fig.savefig(path+'/Spike_Sim/'+"Spike_Sim116_122,124_Error_contour.pdf", bbox_inches='tight', pad_inches=0.1)